In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 18314501
paper_name = 'gustavsson_ronne_2008' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Table1.xlsx', sheet_name='Sheet1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 154 x 2


In [7]:
original_data.head()

,Sensitivity,Gene
0,3,DEF1
1,3,RAD17
2,3,RAD50
3,3,RAD51
4,3,TOP3


In [9]:
original_data['gene'] = original_data['Gene'].astype(str)

In [10]:
# Eliminate all white spaces & capitalize
original_data['gene'] = clean_genename(original_data['gene'])

In [11]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['gene'], to='orf')

In [12]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Sensitivity, Gene, gene, orf]
Index: []


In [13]:
original_data['data'] = -original_data['Sensitivity'].astype(int)

In [14]:
original_data.set_index('orf', inplace=True)

In [15]:
original_data = original_data[['data']].copy()

In [16]:
original_data = original_data.groupby(original_data.index).mean()

In [17]:
original_data.shape

(133, 1)

# Prepare the final dataset

In [18]:
data = original_data.copy()

In [19]:
dataset_ids = [167]
datasets = datasets.reindex(index=dataset_ids)

In [20]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [21]:
data.head()

dataset_id,167
data_type,value
orf,
YBL031W,-3.0
YBL047C,-1.0
YBL079W,-3.0
YBR077C,-2.0
YBR128C,-2.0


## Subset to the genes currently in SGD

In [22]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [23]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,167
,data_type,value
gene_id,orf,
119,YBL031W,-3.0
135,YBL047C,-1.0
167,YBL079W,-3.0
272,YBR077C,-2.0
322,YBR128C,-2.0


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        167          
data_type       value    valuez
gene_id orf                    
119     YBL031W  -3.0 -7.782182
135     YBL047C  -1.0 -2.594061
167     YBL079W  -3.0 -7.782182
272     YBR077C  -2.0 -5.188122
322     YBR128C  -2.0 -5.188122

# Print out

In [27]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db3 import *

In [29]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 18314501...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.08it/s]

Updating the data_modified_on field...
